### Spark Sentence Detector

In [113]:
from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from pyspark.ml import Pipeline
import pandas as pd
import re
import nltk
from nltk import word_tokenize
#pd.set_option("display.max_colwidth", 0)

In [3]:
from pyspark.sql import SparkSession

In [4]:
builder = SparkSession.builder\
        .appName("Spark NLP Licensed")\
        .master("local[*]")\
        .config("spark.driver.memory", "24G")\
        .config("spark.driver.maxResultSize", "2048GB")\
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
        .config("spark.kryoserializer.buffer.max", "2000M")\
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.2")

spark = builder.getOrCreate()

21/12/15 09:34:53 WARN Utils: Your hostname, kurubal resolves to a loopback address: 127.0.1.1; using 192.168.1.37 instead (on interface enp6s0)
21/12/15 09:34:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/kurubal/anaconda3/envs/pyspark_env/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kurubal/.ivy2/cache
The jars for the packages stored in: /home/kurubal/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8d039868-9328-4a2d-97ce-602732a1936b;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.3.2 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lo

In [5]:
spark

In [5]:
#textFiles = spark.sparkContext.wholeTextFiles("oncology_notes/*")

#df = textFiles.toDF(schema=['path','text'])
#df.show()

In [ ]:
sentence_name = "Spark_Sentence_Tokenize1"

In [124]:
df = spark.read.text('/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Pyspark/Temp/xaa.tr').toDF('text') # xaa.txt text data

In [125]:
df.select("text").head(1)

[Row(text='Çeviren: sarkis sarveniko İyi seyirler dileriz.')]

In [126]:
df.rdd.getNumPartitions()

13

In [9]:
df = df.repartition(200)

In [10]:
df.rdd.getNumPartitions()

200

In [127]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
sentencerDL = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")

model = PipelineModel(stages=[documenter, sentencerDL])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]


In [128]:
result = model.transform(df)

In [129]:
result.show()

+--------------------+--------------------+--------------------+
|                text|            document|           sentences|
+--------------------+--------------------+--------------------+
|Çeviren: sarkis s...|[{document, 0, 46...|[{document, 0, 46...|
|Her gün baktığım ...|[{document, 0, 65...|[{document, 0, 65...|
|        TÜRK TUTKUSU|[{document, 0, 11...|[{document, 0, 12...|
|Memnun etmeyi, ba...|[{document, 0, 10...|[{document, 0, 10...|
|Bu açıdan onlara ...|[{document, 0, 32...|[{document, 0, 32...|
|İlk kuzenlerimiz ...|[{document, 0, 38...|[{document, 0, 36...|
|Öyle deme, benim ...|[{document, 0, 37...|[{document, 0, 37...|
|Bu dişiler erkekl...|[{document, 0, 44...|[{document, 0, 44...|
|           Birazcık.|[{document, 0, 8,...|[{document, 0, 8,...|
|     Şu şeker kadar.|[{document, 0, 14...|[{document, 0, 14...|
|Peki, ben nasıl b...|[{document, 0, 28...|[{document, 0, 28...|
|Hiç aptal maymun ...|[{document, 0, 31...|[{document, 0, 31...|
|Şu an kendimi apt...|[{d

In [130]:
result.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentences: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = tru

In [131]:
pd_res = result.select("sentences.result").toPandas()
pd_res

,result
0,[Çeviren: sarkis sarveniko İyi seyirler dileriz.]
1,[Her gün baktığım manzaramın bir mezarlık olacağını kim bilirdi ki?]
2,[TÜRK TUTKUSU]
3,"[Memnun etmeyi, baştan çıkarmayı kandırmayı erkeklerin ne düşündüğünü bilip önceden tahmin etmemiz gerekir.]"
4,[Bu açıdan onlara tahakküm ederiz.]
...,...
1695900,[Ne?]
1695901,[Nedir o?]
1695902,[Dualarına verilen yanıt.]
1695903,"[Hatırlaman gereken tek şey, bunun seninle ilgisi olmadığı.]"


In [132]:
pd_res.to_csv(f"{sentence_name}.csv", index=False)

#### Clean DataFrame Sentence

In [135]:
#!pip install modin[ray]

In [136]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 

2021-12-15 11:24:11,618	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.37',
 'raylet_ip_address': '192.168.1.37',
 'redis_address': '192.168.1.37:6379',
 'object_store_address': '/tmp/ray/session_2021-12-15_11-24-09_515635_11607/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-15_11-24-09_515635_11607/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-12-15_11-24-09_515635_11607',
 'metrics_export_port': 59220,
 'node_id': 'ceb2792ecafdf7eaa8713522489495742fb050ec8d00c21d2c8e999f'}

In [137]:
import modin.pandas as pd

In [138]:
df2 = pd.read_csv(f"{sentence_name}.csv")
df2

,result
0,['Çeviren: sarkis sarveniko İyi seyirler dileriz.']
1,['Her gün baktığım manzaramın bir mezarlık olacağını kim bilirdi ki?']
2,['TÜRK TUTKUSU']
3,"['Memnun etmeyi, baştan çıkarmayı kandırmayı erkeklerin ne düşündüğünü bilip önceden tahmin etmemiz gerekir.']"
4,['Bu açıdan onlara tahakküm ederiz.']
...,...
1695900,['Ne?']
1695901,['Nedir o?']
1695902,['Dualarına verilen yanıt.']
1695903,"['Hatırlaman gereken tek şey, bunun seninle ilgisi olmadığı.']"


In [139]:
def clean_text(text):
    clean_brackets = text.strip("[]")
    clean_brackets_data = re.sub('[\(\[\{].*?[\)\]\}]', '', clean_brackets)
    clean_punc = re.sub(r'[^\w\s]','',clean_brackets_data)
    text_clean_list = re.findall(r'\w+', clean_punc)
    #alpha_num = [w for w in text_clean_list if w.isalnum()]
    text_result = (" ".join(text_clean_list)).lower()
    return text_result 

In [140]:
sent_list = []
for x in df2.result:
    x = clean_text(x)
    sent_list.append(x)  

In [141]:
df_clean = pd.DataFrame(sent_list)
df_clean.head()

,0
0,çeviren sarkis sarveniko i̇yi seyirler dileriz
1,her gün baktığım manzaramın bir mezarlık olacağını kim bilirdi ki
2,türk tutkusu
3,memnun etmeyi baştan çıkarmayı kandırmayı erkeklerin ne düşündüğünü bilip önceden tahmin etmemiz gerekir
4,bu açıdan onlara tahakküm ederiz


In [142]:
df_clean = df_clean.rename(columns={0:"sentence"})
df_clean

,sentence
0,çeviren sarkis sarveniko i̇yi seyirler dileriz
1,her gün baktığım manzaramın bir mezarlık olacağını kim bilirdi ki
2,türk tutkusu
3,memnun etmeyi baştan çıkarmayı kandırmayı erkeklerin ne düşündüğünü bilip önceden tahmin etmemiz gerekir
4,bu açıdan onlara tahakküm ederiz
...,...
1695900,ne
1695901,nedir o
1695902,dualarına verilen yanıt
1695903,hatırlaman gereken tek şey bunun seninle ilgisi olmadığı


In [143]:
df_clean.drop(df_clean[df_clean.sentence == ""].index, inplace=True)

In [144]:
sent_count = df_clean.sentence.value_counts().sort_values(ascending=False)
#sent_count

In [145]:
sent_count_df = pd.DataFrame(sent_count).reset_index()
sent_count_df.rename(columns={"index":"sentence", "sentence":"frequency"}, inplace=True)

In [146]:
total_frequency = sent_count_df.frequency.sum()

In [147]:
sent_count_df["ratio"] = (sent_count_df.frequency/total_frequency)*100

In [148]:
def sentence_lenght(sentence):
    word_var2 = re.findall(r'\w+', sentence)
    if len(word_var2) <= 10:
        return sentence
    else:
        return "sentence is bigger than ten word"

In [149]:
sent_count_df.sentence = sent_count_df.sentence.apply(sentence_lenght)

In [150]:
sent_count_df.drop(sent_count_df[sent_count_df.sentence == "sentence is bigger than ten word"].index, axis=0, inplace=True)
sent_count_df.reset_index(drop=True, inplace=True)

In [151]:
df_result = sent_count_df[sent_count_df.frequency>1]

In [152]:
df_result.to_csv("Clean_Spark_Sentence_Tokenize1.csv", index=False)